# Importaciones

In [1]:
import json 
import datetime
import re

import pandas as pd 
from googleapiclient.errors import HttpError
from Google import Create_Service

Función que utiliza expresiones regulares para obtener los segundos totales de una duración dada.

In [6]:
def convert_duration(duration):
    try:
        h = int(re.search('\d+H', duration)[0][:-1]) * 60**2  if re.search('\d+H', duration) else 0 # hour
        m = int(re.search('\d+M', duration)[0][:-1]) * 60  if re.search('\d+M', duration) else 0 # minute
        s = int(re.search('\d+S', duration)[0][:-1])  if re.search('\d+S', duration) else 0 # second
        return h + m + s
    except Exception as e:
        print(e)
        return 0 

In [22]:
def retrieve_playlists(service, channelId):
    playlists =[]
    try:
        response = service.playlists().list(
            part = 'contentDetails, snippet, status',
            channelId = channelId,
            maxResults = 50
        ).execute()

        playlists.extend(response.get('items'))
        nextPageToken = response.get('nextPageToken')

        while nextPageToken:
            response = service.playlists().list(
                part = 'contentDetails, snippet, status',
                channelId = channelId,
                maxResults = 50,
                pageToken = nextPageToken
            ).execute()

            playlists.extend(response.get('items'))
            nextPageToken = response.get('nextPageToken')

        return playlists

    except HttpError as e:
        errMsg = json.loads(e.content)
        print("HTTP Error: ")
        print(errMsg['error']['message'])
        return []

In [70]:
def retrieve_playlists_items(service, playlistId):
    items =[]
    try:
        response = service.playlistItems().list(
            part = 'contentDetails, snippet, status',
            playlistId = playlistId,
            maxResults = 5
        ).execute()

        items.extend(response.get('items'))
        nextPageToken = response.get('nextPageToken')

        while nextPageToken:
            response = service.playlistItems().list(
                part = 'contentDetails, snippet, status',
                playlistId = playlistId,
                maxResults = 5,
                pageToken = nextPageToken
            ).execute()

            items.extend(response.get('items'))
            nextPageToken = response.get('nextPageToken')

        return items

    except HttpError as e:
        errMsg = json.loads(e.content)
        print("HTTP Error: ")
        print(errMsg['error']['message'])
        return []

Creando constantes para llamar a la API

In [17]:
CLIENT_SECRETS_FILE = 'client_secret.json'
API_NAME = 'youtube'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/youtube.readonly']

service = Create_Service(CLIENT_SECRETS_FILE, API_NAME, API_VERSION, SCOPES)

client_secret.json-youtube-v3-(['https://www.googleapis.com/auth/youtube.readonly'],)
['https://www.googleapis.com/auth/youtube.readonly']
youtube service created successfully


El objeto <code>service</code> nos da accesos a todos los recursos de la API

In [18]:
dir(service)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_basic_methods',
 '_add_nested_resources',
 '_add_next_methods',
 '_baseUrl',
 '_developerKey',
 '_dynamic_attrs',
 '_http',
 '_model',
 '_requestBuilder',
 '_resourceDesc',
 '_rootDesc',
 '_schema',
 '_set_dynamic_attr',
 '_set_service_methods',
 'abuseReports',
 'activities',
 'captions',
 'channelBanners',
 'channelSections',
 'channels',
 'close',
 'commentThreads',
 'comments',
 'i18nLanguages',
 'i18nRegions',
 'liveBroadcasts',
 'liveChatBans',
 'liveChatMessages',
 'liveChatModerators',
 'liveStreams',
 'members',
 'membershipsLevels',
 'new_batch_h

Ahora llamemos a la fución que nos provee de las playlists

In [58]:
channel_id = 'UCxEgOKuI-n-WOJaNcisHvSg' # ID del canal de el presidente
playlists = retrieve_playlists(service, channel_id)

Obtenemos la playlist de las conferencias matutinas

In [43]:
playlists[-1]

{'kind': 'youtube#playlist', 'etag': 'yFdEtWGvpytKuVfVI0pO98jWwEE', 'id': 'PLRnlRGar-_296KTsVL0R6MEbpwJzD8ppA', 'snippet': {'publishedAt': '2019-03-08T21:50:49Z', 'channelId': 'UCxEgOKuI-n-WOJaNcisHvSg', 'title': 'Conferencias de prensa matutinas', 'description': 'De lunes a viernes, a las 7:00 horas, mantenemos un diálogo circular con representantes de los medios de comunicación al término de la reunión con Gabinete de Seguridad.', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/uW-UISq5YLY/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/uW-UISq5YLY/mqdefault.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/uW-UISq5YLY/hqdefault.jpg', 'width': 480, 'height': 360}, 'standard': {'url': 'https://i.ytimg.com/vi/uW-UISq5YLY/sddefault.jpg', 'width': 640, 'height': 480}, 'maxres': {'url': 'https://i.ytimg.com/vi/uW-UISq5YLY/maxresdefault.jpg', 'width': 1280, 'height': 720}}, 'channelTitle': 'Andrés Manuel López Obrador

In [71]:
playlistId = 'PLRnlRGar-_296KTsVL0R6MEbpwJzD8ppA'
playlists_items = retrieve_playlists_items(service, playlistId)

In [73]:
playlists_items[0]

{'kind': 'youtube#playlistItem', 'etag': '8WJEXuec_liHMLQ3MWFIqoEQmWs', 'id': 'UExSbmxSR2FyLV8yOTZLVHNWTDBSNk1FYnB3SnpEOHBwQS4zRDNFM0Q1MzY2NTYwMTJC', 'snippet': {'publishedAt': '2023-04-24T12:44:41Z', 'channelId': 'UCxEgOKuI-n-WOJaNcisHvSg', 'title': 'Conferencia de prensa matutina desde Palacio Nacional. Lunes 24 de abril 2023 | Presidente AMLO', 'description': 'Conferencia de prensa matutina, desde Palacio Nacional. Lunes 24 de abril 2023 | Presidente AMLO.\n\n0:00 Inicio de transmisión\n15:15 Comienza la conferencia de prensa del presidente Andrés Manuel López Obrador\n16:48 Comparativo de marcas y precios de combustibles\n25:07 Comparativo de marcas y precios de productos de primera necesidad\n28:30 Reporte de construcción del Tren Maya en tramo Escárcega-Calkiní\n32:57 Informe del consorcio Carso FCC en el Tramo 2 del Tren Maya\n34:14 Salvamento arqueológico en el proyecto del Tren Maya\n40:26 Informe de la Semarnat en el Tramo 2 del Tren Maya\n42:33 Informe de la Secretaría del B